In [57]:
class Person(object):
    def __init__(self, name, age, gender):
        self.name = name
        self.age = age
        self.gender = gender
        if self.age < 0 :
            self.gender = 'ValueError'
        

In [58]:
import copy
import csv
import os
import zipfile
class Josephus:
    '约瑟夫环'

    def __init__(self, reader = None):
        self._people = []
        if reader:
            for each in reader:
                self._people.append(each)
        self.step = 0
        self.start = 1


    def append(self, obj):
        self._people.append(obj)

    def pop(self):
        self._people.pop(0)

    def query_list(self):
        return self._people

    def __len__(self):
        return len(self._people)

    def next_bymod(self):
        temp = copy.copy(self._people)
        length = len(temp)
        if length == 0:
            return None
        id_ = self.start

        for i in range(length):
            id_ = (id_ + self.step - 1) % len(temp)
            index = temp.pop(id_)
            yield index

    def next_circular(self):
        temp = copy.copy(self._people)
        if len(temp) == 0:
            return None

        index = 1
        start = self.start % len(temp)
        temp = temp[start:] + temp[:start]

        while index:
            head = (self.step-1) % len(temp)
            temp = temp[head:] + temp[:head]
            index = temp.pop(0)
            yield index



In [59]:
class Reader:
    def create_person_from_file(self):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def close_file(self):
        raise NotImplementedError

class TxtReader(Reader):
    def __init__(self, path):
        self.file = open(path, 'r', encoding='utf-8')
        self._all_data = self.file.readlines()

    def create_person_from_file(self):
        for each in self._all_data:
            data = each.strip().replace(' ','').split(',')
            name = data[0]
            try:
                age = int(data[1])
            except ValueError as e:
                age = 'ValueError'
            gender = data[2]
            yield Person(name, age, gender)

    def __len__(self):
        return len(self._all_data)

    def close_file(self):
        self.file.close()
        
class CsvReader(Reader):
    def __init__(self, path):
        self.file = open(path, 'r', encoding='gbk')
        self._all_data = self.file.readlines()

    def create_person_from_file(self):
        for each in self._all_data:
            data = each.strip().replace(' ','').split(',')
            name = data[0]
            try:
                age = int(data[1])
            except ValueError as e:
                age = -1
            gender = data[2]
            yield Person(name, age, gender)

    def __len__(self):
        return len(self._all_data)

    def close_file(self):
        self.file.close()

class ZipReader(Reader):
    def __init__(self, path, file_name):
        with zipfile.ZipFile(path, 'r') as zip_file:
            file_list = zip_file.namelist()
            if file_name not in file_list:
                raise FileNotFoundError

            file_path = zip_file.extract(file_name)
            file_type = file_name.split('.')[1]
            if file_type == 'txt':
                encode_standard = 'utf-8'
            if file_type == 'csv':
                encode_standard = 'gbk'

            self.file = open(file_path, 'r', encoding = encode_standard)
            self._all_data = self.file.readlines()

    def create_person_from_file(self):
        for each in self._all_data:
            data = each.strip().replace(' ','').split(',')
            name = data[0]
            try:
                age = int(data[1])
            except ValueError as e:
                age = 'ValueError'
            gender = data[2]
            yield Person(name, age, gender)
            
    def __len__(self):
        return len(self._all_data)

    def close_file(self):
        self.file.close()


In [61]:
path = 'people.txt'
file_type = path.split('.')[1]
if (file_type != 'txt') and (file_type != 'csv') and (file_type != 'zip'):
    raise Exception('文件类型错误')

read_file = TxtReader(path)
conductor = read_file.create_person_from_file()

file_length = len(read_file)
print(file_length)

reader = []
for i in range(file_length):
    reader.append(next(conductor))


ring = Josephus(reader)
ring.start = 0
ring.step = 2

ring_length = len(ring)

generator_bymod = ring.next_bymod()
for i in range(ring_length):
    someone = next(generator_bymod)
    print("第{}个出列的人:{}；年龄:{}；性别:{}". format(i+1, someone.name, someone.age, someone.gender))

print('-'*30)

generator_circular = ring.next_circular()
for i in range(ring_length):
    someone = next(generator_circular)
    print("第{}个出列的人:{}；年龄:{}；性别:{}". format(i+1, someone.name, someone.age, someone.gender))


5
第1个出列的人:Rick；年龄:53；性别:男
第2个出列的人:Beth；年龄:34；性别:女
第3个出列的人:Morty；年龄:12；性别:男
第4个出列的人:Jerry；年龄:36；性别:男
第5个出列的人:Summer；年龄:16；性别:女
------------------------------
第1个出列的人:Rick；年龄:53；性别:男
第2个出列的人:Beth；年龄:34；性别:女
第3个出列的人:Morty；年龄:12；性别:男
第4个出列的人:Jerry；年龄:36；性别:男
第5个出列的人:Summer；年龄:16；性别:女


In [62]:
path = 'people.zip'
file_name = 'people.csv'
read_file = ZipReader(path, file_name)
conductor = read_file.create_person_from_file()

length = len(read_file)
assert(length == 5)

for i in range(length):
    print(next(conductor).name)

Morty
Rick
Summer
Beth
Jerry
